# 1. Install and Import Dependencies

In [ ]:
!pip install mediapipe
import mediapipe as mp
!pip install opencv-python
import cv2
!pip install numpy
import numpy as np
import uuid
import os
import time
import random
import shutil
!pip install pandas
!pip install pyarrow
import pandas as pd

# 2. Take Output Images (100 images for each category)

In [ ]:
# Initialisation of MediaPipe and Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# do not modify parameters
hands = mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) 

# set category of current images and create folder
# {zero, one, two, three, four, five, spock, other}
imageCategory = "other"
os.makedirs(imageCategory, exist_ok=True) 

cap = cv2.VideoCapture(0)

image_count = 0  

while cap.isOpened() and image_count < 100:  # take 100 images
    ret, frame = cap.read()
    time.sleep(0.5)
    
    # BGR 2 RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    image = cv2.flip(image, 1)
    
    image.flags.writeable = False
    
    # Detections
    results = hands.process(image)

    image.flags.writeable = True
    
    # RGB 2 BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    # create copy of image (before landmarks are drawn) which can be saved later
    image_copy = image.copy()
    
    # Rendering results
    if results.multi_hand_landmarks:
        for num, hand in enumerate(results.multi_hand_landmarks):
            mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                    mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                    mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
                                     )
    
    # Save the image without landmarks if recognized
    if results.multi_hand_landmarks:
        image_count += 1
        image_filename = f'{imageCategory}/image_{imageCategory}_{image_count}.jpg'
        cv2.imwrite(image_filename, image_copy)
    
    cv2.imshow('Hand Tracking', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# 3. Rename And Divide Into Test (80%), Train(10%), Validation(10%)

In [ ]:
# Source and destination folder paths
source_folder =  r"C:/Users/henri/Documents/FingerCounterData/DatasetForFingerCounter/Dominik_other" # Replace this with your source folder path

training_folder = r"C:/Users/henri/Documents/FingerCounterData/DatasetForFingerCounter/Training"  # Replace this with your destination folder path
test_folder = r"C:/Users/henri/Documents/FingerCounterData/DatasetForFingerCounter/Test"  # Replace this with your destination folder path
validation_folder = r"C:/Users/henri/Documents/FingerCounterData/DatasetForFingerCounter/Validation"  # Replace this with your destination folder path

# Prefix to be added
prefix = "Dominik"

# Function to move pictures from each subfolder to the destination folder with a new prefix
def move_pictures_with_prefixes(folder_path):
    picture_files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png', '.gif'))]
    
    random.shuffle(picture_files)

    total_pictures = len(picture_files)
    # 80% are training data
    training_80_percent = int(total_pictures * 0.8)
    # 10% are test data
    test_10_percent = int(total_pictures * 0.1)
    # 10% are verification data
    validation_10_percent = int(total_pictures * 0.1)

    percentages = [training_80_percent, test_10_percent, validation_10_percent, total_pictures]
    folders = [training_folder, test_folder, validation_folder]
    
    for i, percentage in enumerate(percentages[:-1]):
        destination_folder = folders[i]

        selected_files = picture_files[:percentage]
        picture_files = picture_files[percentage:]

        for file_name in selected_files:
            file_path = os.path.join(folder_path, file_name)
            new_file_name = f"{prefix}_{file_name}"
            destination_path = os.path.join(destination_folder, new_file_name)
            shutil.move(file_path, destination_path)


move_pictures_with_prefixes(source_folder)


# 4. Create Landmarks And Write Them In CSV Files

In [ ]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5)

# Set the path to the directory containing the images
path = r'C:\Users\henri\Documents\FingerCounterData\DatasetForFingerCounter'

# Loop through each subdirectory in the directory
for subdir in os.listdir(path):
    data = []
    subdir_path = os.path.join(path, subdir)
    
    if os.path.isdir(subdir_path): 
        if(('.git' not in subdir_path)):  
            for filename in os.listdir(subdir_path):     
                if filename.endswith('.jpg'):
                    image_path = os.path.join(subdir_path, filename)
                    
                    # Read the image using cv2.imread()
                    image = cv2.imread(image_path)
                    # Convert the image from BGR to RGB using cv2.cvtColor()
                    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    # Process the image using the hands.process() method
                    results = hands.process(image_rgb)
                    # Extract the landmarks from the processed image
                    landmarks = results.multi_hand_landmarks
                    if landmarks:
                        # Extract the label from the subdirectory name
                        # Create a dictionary containing the label and the landmarks
                        
                        landmark_dict = {'filename': filename}
                        for i, landmark in enumerate(landmarks[0].landmark):
                            landmark_dict[f'x{i}'] = landmark.x
                            landmark_dict[f'y{i}'] = landmark.y
                            landmark_dict[f'z{i}'] = landmark.z
                        # Append the dictionary to the data list
                        data.append(landmark_dict)
                        print(data)
            df = pd.DataFrame(data)
            csv_path = os.path.join(path,subdir + '_landmarks.csv')
            df.to_csv(csv_path, index=False)



# 5. Create copy of all images with their corresponding landmarks from the csv files

In [ ]:
''' for each image category one folder is created in which all images of this category are copied with their drawn landmarks '''
''' images with no landmarks are also copied in new folders '''

#needed to reconstruct landmarks from coordinates
from mediapipe.framework.formats import landmark_pb2

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

images_folders = [r"C:\Users\leons\Documents\DeepLearning\DatasetForFingerCounter\Test", 
                  r"C:\Users\leons\Documents\DeepLearning\DatasetForFingerCounter\Training", 
                  r"C:\Users\leons\Documents\DeepLearning\DatasetForFingerCounter\Validation"]
csv_files = [r"C:\Users\leons\Documents\DeepLearning\DatasetForFingerCounter\Test_landmarks.csv", 
             r"C:\Users\leons\Documents\DeepLearning\DatasetForFingerCounter\Training_landmarks.csv", 
             r"C:\Users\leons\Documents\DeepLearning\DatasetForFingerCounter\Validation_landmarks.csv"]
class_names = ["zero", "one", "two", "three", "four", "five", "spock", "other"]

for name in class_names:
    
    output_folder = f"{name}_wrong_images"
    os.makedirs(output_folder, exist_ok=True)  # Create the folder if it doesn't exist
    
    for image_folder, csv_file in zip(images_folders, csv_files): 
        file_names = []
        #fill the array with all image names for current folder and cat name
        for file in os.listdir(image_folder):
            if name in file:
                file_names.append(file)
        
        landmarks_df = pd.read_csv(csv_file, header=0)

        for file_name in file_names:
            #extract the landmarks of one image
            
            #extract the only the row of the current file in the csv
            specific_row = landmarks_df[landmarks_df['filename'] == file_name]

            #read the image
            file_path = os.path.join(image_folder, file_name)
            frame = cv2.imread(file_path)
            
            if not specific_row.empty:
                
                #drop the "filename" column
                landmarks = specific_row.drop('filename', axis=1).values[0]
                #reshape into 21*3 array
                landmarks_array = np.array(landmarks).reshape(21, 3)
            
                #reshape the coordinates from landmarks_array into actual NormalizedLandmarks objects
                hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
                for landmark_coords in landmarks_array:
                    landmark_proto = landmark_pb2.NormalizedLandmark()
                    landmark_proto.x = landmark_coords[0]
                    landmark_proto.y = landmark_coords[1]
                    landmark_proto.z = landmark_coords[2]
                    hand_landmarks_proto.landmark.append(landmark_proto)
                
                mp_drawing.draw_landmarks(frame, hand_landmarks_proto, mp_hands.HAND_CONNECTIONS,
                                                  mp_drawing.DrawingSpec(color=(20, 200, 20), thickness=2, circle_radius=4),
                                                  mp_drawing.DrawingSpec(color=(20, 20, 200), thickness=2, circle_radius=2))
                
                #indent following code here > pictures with no landmarks are not processed
            # Save the modified image in the output folder
            output_file_path = os.path.join(output_folder, f"{file_name[:-4]}.jpg")
            cv2.imwrite(output_file_path, frame)


# 6. user should manually revise all images and delete the ones with wrong landmarks

# 7. clear csv files (delete csv datasets of previousely deleted images) and delete images with no landmarks

In [ ]:
images_folders = [r"C:\Users\leons\Documents\DeepLearning\DatasetForFingerCounter\Test", 
                  r"C:\Users\leons\Documents\DeepLearning\DatasetForFingerCounter\Training", 
                  r"C:\Users\leons\Documents\DeepLearning\DatasetForFingerCounter\Validation"]
csv_files = [r"C:\Users\leons\Documents\DeepLearning\DatasetForFingerCounter\Test_landmarks.csv", 
             r"C:\Users\leons\Documents\DeepLearning\DatasetForFingerCounter\Training_landmarks.csv", 
             r"C:\Users\leons\Documents\DeepLearning\DatasetForFingerCounter\Validation_landmarks.csv"]

# loop through csv files and delete all csv datasets that have no corresponding image (csv datasets of images that have been deleted manually before)
for image_folder, csv_file in zip(images_folders, csv_files):
        images_names = os.listdir(image_folder)
        data = pd.read_csv(csv_file, header=0)
        filtered_data = data[data['filename'].isin(images_names)]
        filtered_data.to_csv(csv_file, index=False)


# loop through images and delete all images that have no corresponding csv datasets (images that where no landmarks could be detected)
for image_folder, csv_file in zip(images_folders, csv_files):
        images_names = os.listdir(image_folder)
        data = pd.read_csv(csv_file, header=0)

        # Get the filenames present in the CSV
        csv_image_names = data['filename'].tolist()

        # Delete images not in the CSV
        for image_name in images_names:
            if image_name not in csv_image_names:
                os.remove(os.path.join(image_folder, image_name))
                

# ONLY FOR TEST PURPOSE: Read One Image and Create And Display Landmarks

In [ ]:
# Load the image file
file_path = r"C:\Users\leons\Documents\DeepLearning\DatasetForFingerCounter\Verification\Alex_image_spock_45.jpg"  # Replace this with the path to image file
frame = cv2.imread(file_path)

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands: 
   
        # BGR 2 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Set flag
        image.flags.writeable = False
        
        # Detections
        results = hands.process(image)
        
        # Set flag to true
        image.flags.writeable = True
        
        # RGB 2 BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Detections
        print(results.multi_hand_landmarks)
    
     
        # Rendering results
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                        mp_drawing.DrawingSpec(color=(20, 200, 20), thickness=2, circle_radius=4),
                                        mp_drawing.DrawingSpec(color=(20, 20, 200), thickness=2, circle_radius=2),
                                         )
        
        
        cv2.imshow('Hand Tracking', image)
        cv2.waitKey(0)

cv2.destroyAllWindows()